# Aandachtsmechanismen en transformers

Een belangrijk nadeel van recurrente netwerken is dat alle woorden in een reeks dezelfde invloed hebben op het resultaat. Dit leidt tot suboptimale prestaties bij standaard LSTM encoder-decoder modellen voor sequentie-tot-sequentie taken, zoals Named Entity Recognition en Machine Translation. In werkelijkheid hebben specifieke woorden in de invoerreeks vaak meer invloed op de opeenvolgende uitvoer dan andere.

Laten we een sequentie-tot-sequentie model overwegen, zoals machinevertaling. Dit wordt geïmplementeerd door twee recurrente netwerken, waarbij één netwerk (**encoder**) de invoerreeks samenvat in een verborgen toestand, en een ander netwerk, **decoder**, deze verborgen toestand uitrolt naar het vertaalde resultaat. Het probleem met deze aanpak is dat de eindtoestand van het netwerk moeite heeft om het begin van een zin te onthouden, wat leidt tot een slechte kwaliteit van het model bij lange zinnen.

**Aandachtsmechanismen** bieden een manier om het contextuele effect van elke invoervector op elke uitvoervoorspelling van het RNN te wegen. Dit wordt geïmplementeerd door shortcuts te creëren tussen de tussenliggende toestanden van het invoer-RNN en het uitvoer-RNN. Op deze manier houden we bij het genereren van het uitvoersymbool $y_t$ rekening met alle verborgen toestanden van de invoer $h_i$, met verschillende gewichtscoëfficiënten $\alpha_{t,i}$.

![Afbeelding van een encoder/decoder model met een additieve aandachtlaag](../../../../../translated_images/nl/encoder-decoder-attention.7a726296894fb567.webp)
*Het encoder-decoder model met additief aandachtsmechanisme in [Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf), geciteerd uit [deze blogpost](https://lilianweng.github.io/lil-log/2018/06/24/attention-attention.html)*

De aandachtsmatrix $\{\alpha_{i,j}\}$ vertegenwoordigt de mate waarin bepaalde invoerwoorden een rol spelen bij het genereren van een bepaald woord in de uitvoerreeks. Hieronder staat een voorbeeld van zo'n matrix:

![Afbeelding van een voorbeelduitlijning gevonden door RNNsearch-50, afkomstig van Bahdanau - arviz.org](../../../../../translated_images/nl/bahdanau-fig3.09ba2d37f202a6af.webp)

*Afbeelding afkomstig uit [Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf) (Fig.3)*

Aandachtsmechanismen zijn verantwoordelijk voor veel van de huidige of bijna huidige state-of-the-art in Natural Language Processing. Het toevoegen van aandacht verhoogt echter aanzienlijk het aantal modelparameters, wat leidde tot schaalproblemen met RNN's. Een belangrijke beperking van het schalen van RNN's is dat de recurrente aard van de modellen het uitdagend maakt om training te batchen en te paralleliseren. In een RNN moet elk element van een reeks in volgorde worden verwerkt, wat betekent dat het niet gemakkelijk parallel kan worden uitgevoerd.

De adoptie van aandachtsmechanismen in combinatie met deze beperking leidde tot de creatie van de huidige state-of-the-art Transformer-modellen die we vandaag kennen en gebruiken, van BERT tot OpenGPT3.

## Transformer-modellen

In plaats van de context van elke vorige voorspelling door te geven aan de volgende evaluatiestap, gebruiken **transformer-modellen** **positionele encoderingen** en aandacht om de context van een gegeven invoer vast te leggen binnen een opgegeven tekstvenster. De onderstaande afbeelding laat zien hoe positionele encoderingen met aandacht context kunnen vastleggen binnen een bepaald venster.

![Geanimeerde GIF die laat zien hoe de evaluaties worden uitgevoerd in transformer-modellen.](../../../../../lessons/5-NLP/18-Transformers/images/transformer-animated-explanation.gif)

Omdat elke invoerpositie onafhankelijk wordt gemapt naar elke uitvoerpositie, kunnen transformers beter paralleliseren dan RNN's, wat veel grotere en expressievere taalmodellen mogelijk maakt. Elke aandachtshoofd kan worden gebruikt om verschillende relaties tussen woorden te leren, wat downstream Natural Language Processing-taken verbetert.

**BERT** (Bidirectional Encoder Representations from Transformers) is een zeer groot meerlagig transformer-netwerk met 12 lagen voor *BERT-base* en 24 voor *BERT-large*. Het model wordt eerst voorgetraind op een grote corpus van tekstdata (Wikipedia + boeken) met behulp van ongesuperviseerde training (voorspellen van gemaskeerde woorden in een zin). Tijdens het voortrainen absorbeert het model een significant niveau van taalbegrip, dat vervolgens kan worden benut met andere datasets door middel van fine-tuning. Dit proces wordt **transfer learning** genoemd.

![Afbeelding van http://jalammar.github.io/illustrated-bert/](../../../../../translated_images/nl/jalammarBERT-language-modeling-masked-lm.34f113ea5fec4362.webp)

Er zijn veel variaties van Transformer-architecturen, waaronder BERT, DistilBERT, BigBird, OpenGPT3 en meer, die kunnen worden gefinetuned. Het [HuggingFace-pakket](https://github.com/huggingface/) biedt een repository voor het trainen van veel van deze architecturen met PyTorch.

## BERT gebruiken voor tekstanalyse

Laten we eens kijken hoe we een voorgetraind BERT-model kunnen gebruiken om onze traditionele taak op te lossen: sequentieclassificatie. We gaan onze oorspronkelijke AG News-dataset classificeren.

Laten we eerst de HuggingFace-bibliotheek en onze dataset laden:


In [10]:
import torch
import torchtext
from torchnlp import *
import transformers
train_dataset, test_dataset, classes, vocab = load_dataset()
vocab_len = len(vocab)

Loading dataset...
Building vocab...


Omdat we een vooraf getraind BERT-model gaan gebruiken, moeten we een specifieke tokenizer gebruiken. Eerst laden we een tokenizer die gekoppeld is aan het vooraf getrainde BERT-model.

De HuggingFace-bibliotheek bevat een repository met vooraf getrainde modellen, die je kunt gebruiken door simpelweg hun namen als argumenten op te geven bij de `from_pretrained` functies. Alle benodigde binaire bestanden voor het model worden automatisch gedownload.

Echter, soms moet je je eigen modellen laden. In dat geval kun je de directory specificeren die alle relevante bestanden bevat, inclusief parameters voor de tokenizer, het `config.json`-bestand met modelparameters, binaire gewichten, enzovoort.


In [11]:
# To load the model from Internet repository using model name. 
# Use this if you are running from your own copy of the notebooks
bert_model = 'bert-base-uncased' 

# To load the model from the directory on disk. Use this for Microsoft Learn module, because we have
# prepared all required files for you.
bert_model = './bert'

tokenizer = transformers.BertTokenizer.from_pretrained(bert_model)

MAX_SEQ_LEN = 128
PAD_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
UNK_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.unk_token)

Het `tokenizer` object bevat de `encode` functie die direct kan worden gebruikt om tekst te coderen:


In [15]:
tokenizer.encode('PyTorch is a great framework for NLP')

[101, 1052, 22123, 2953, 2818, 2003, 1037, 2307, 7705, 2005, 17953, 2361, 102]

Laten we vervolgens iterators maken die we tijdens de training zullen gebruiken om toegang te krijgen tot de gegevens. Omdat BERT zijn eigen coderingsfunctie gebruikt, moeten we een opvulfunctie definiëren die vergelijkbaar is met `padify` die we eerder hebben gedefinieerd:


In [4]:
def pad_bert(b):
    # b is the list of tuples of length batch_size
    #   - first element of a tuple = label, 
    #   - second = feature (text sequence)
    # build vectorized sequence
    v = [tokenizer.encode(x[1]) for x in b]
    # compute max length of a sequence in this minibatch
    l = max(map(len,v))
    return ( # tuple of two tensors - labels and features
        torch.LongTensor([t[0] for t in b]),
        torch.stack([torch.nn.functional.pad(torch.tensor(t),(0,l-len(t)),mode='constant',value=0) for t in v])
    )

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=8, collate_fn=pad_bert, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=8, collate_fn=pad_bert)

In ons geval zullen we een voorgetraind BERT-model genaamd `bert-base-uncased` gebruiken. Laten we het model laden met behulp van het pakket `BertForSequenceClassification`. Dit zorgt ervoor dat ons model al de vereiste architectuur voor classificatie heeft, inclusief de uiteindelijke classifier. Je zult een waarschuwingsbericht zien waarin staat dat de gewichten van de uiteindelijke classifier niet zijn geïnitialiseerd en dat het model pre-training nodig heeft - dat is helemaal prima, want dat is precies wat we gaan doen!


In [9]:
model = transformers.BertForSequenceClassification.from_pretrained(bert_model,num_labels=4).to(device)

Some weights of the model checkpoint at ./bert were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ./bert and

Nu kunnen we beginnen met trainen! Omdat BERT al vooraf getraind is, willen we starten met een vrij kleine leersnelheid om de initiële gewichten niet te beschadigen.

Het zware werk wordt gedaan door het `BertForSequenceClassification` model. Wanneer we het model toepassen op de trainingsdata, geeft het zowel verlies als netwerkoutput terug voor de input minibatch. We gebruiken het verlies voor parameteroptimalisatie (`loss.backward()` voert de backward pass uit), en `out` om de trainingsnauwkeurigheid te berekenen door de verkregen labels `labs` (berekend met `argmax`) te vergelijken met de verwachte `labels`.

Om het proces te controleren, accumuleren we verlies en nauwkeurigheid over meerdere iteraties en printen we deze elke `report_freq` trainingscycli.

Deze training zal waarschijnlijk behoorlijk wat tijd in beslag nemen, dus we beperken het aantal iteraties.


In [6]:
optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)

report_freq = 50
iterations = 500 # make this larger to train for longer time!

model.train()

i,c = 0,0
acc_loss = 0
acc_acc = 0

for labels,texts in train_loader:
    labels = labels.to(device)-1 # get labels in the range 0-3         
    texts = texts.to(device)
    loss, out = model(texts, labels=labels)[:2]
    labs = out.argmax(dim=1)
    acc = torch.mean((labs==labels).type(torch.float32))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    acc_loss += loss
    acc_acc += acc
    i+=1
    c+=1
    if i%report_freq==0:
        print(f"Loss = {acc_loss.item()/c}, Accuracy = {acc_acc.item()/c}")
        c = 0
        acc_loss = 0
        acc_acc = 0
    iterations-=1
    if not iterations:
        break

Loss = 1.1254194641113282, Accuracy = 0.585
Loss = 0.6194715118408203, Accuracy = 0.83
Loss = 0.46665248870849607, Accuracy = 0.8475
Loss = 0.4309701919555664, Accuracy = 0.8575
Loss = 0.35427074432373046, Accuracy = 0.8825
Loss = 0.3306886291503906, Accuracy = 0.8975
Loss = 0.30340143203735354, Accuracy = 0.8975
Loss = 0.26139299392700194, Accuracy = 0.915
Loss = 0.26708646774291994, Accuracy = 0.9225
Loss = 0.3667240524291992, Accuracy = 0.8675


Je kunt zien (vooral als je het aantal iteraties verhoogt en lang genoeg wacht) dat BERT-classificatie ons een behoorlijk goede nauwkeurigheid geeft! Dat komt omdat BERT de structuur van de taal al vrij goed begrijpt, en we alleen de uiteindelijke classifier hoeven te fine-tunen. Echter, omdat BERT een groot model is, duurt het hele trainingsproces lang en vereist het serieuze rekenkracht! (GPU, en bij voorkeur meer dan één).

> **Note:** In ons voorbeeld hebben we een van de kleinste voorgetrainde BERT-modellen gebruikt. Er zijn grotere modellen die waarschijnlijk betere resultaten opleveren.


## Het evalueren van de modelprestaties

Nu kunnen we de prestaties van ons model evalueren op de testdataset. De evaluatielus lijkt sterk op de trainingslus, maar we mogen niet vergeten om het model in evaluatiemodus te zetten door `model.eval()` aan te roepen.


In [10]:
model.eval()
iterations = 100
acc = 0
i = 0
for labels,texts in test_loader:
    labels = labels.to(device)-1      
    texts = texts.to(device)
    _, out = model(texts, labels=labels)[:2]
    labs = out.argmax(dim=1)
    acc += torch.mean((labs==labels).type(torch.float32))
    i+=1
    if i>iterations: break
        
print(f"Final accuracy: {acc.item()/i}")

Final accuracy: 0.9047029702970297


## Belangrijkste punten

In deze unit hebben we gezien hoe eenvoudig het is om een voorgetraind taalmodel uit de **transformers**-bibliotheek te gebruiken en aan te passen voor onze tekstclassificatietaak. Op dezelfde manier kunnen BERT-modellen worden gebruikt voor entiteitsextractie, vraag-antwoord systemen en andere NLP-taken.

Transformermodellen vertegenwoordigen de huidige state-of-the-art in NLP, en in de meeste gevallen zou dit de eerste oplossing moeten zijn waarmee je begint te experimenteren bij het implementeren van aangepaste NLP-oplossingen. Het begrijpen van de basisprincipes van recurrente neurale netwerken, zoals besproken in deze module, is echter van cruciaal belang als je geavanceerde neurale modellen wilt bouwen.



---

**Disclaimer**:  
Dit document is vertaald met behulp van de AI-vertalingsservice [Co-op Translator](https://github.com/Azure/co-op-translator). Hoewel we streven naar nauwkeurigheid, dient u zich ervan bewust te zijn dat geautomatiseerde vertalingen fouten of onnauwkeurigheden kunnen bevatten. Het originele document in zijn oorspronkelijke taal moet worden beschouwd als de gezaghebbende bron. Voor cruciale informatie wordt professionele menselijke vertaling aanbevolen. Wij zijn niet aansprakelijk voor eventuele misverstanden of verkeerde interpretaties die voortvloeien uit het gebruik van deze vertaling.
